In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Load pictures and labels:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical

def load_pictures(link):
    with open(link, 'rb') as f:
        f.read(16)
        buffer = f.read()
        pictures = np.frombuffer(buffer, dtype=np.uint8).reshape(-1, 28, 28)
    return pictures

def load_labels(link):
    with open(link, 'rb') as f:
        f.read(8)
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

train_images = load_pictures('/content/drive/MyDrive/MÁSTER/IC/P1/Train/train-images.idx3-ubyte')
train_labels = load_labels('/content/drive/MyDrive/MÁSTER/IC/P1/Train/train-labels.idx1-ubyte')
evaluation_images = load_pictures('/content/drive/MyDrive/MÁSTER/IC/P1/Evaluation/t10k-images.idx3-ubyte')
evaluation_labels = load_labels('/content/drive/MyDrive/MÁSTER/IC/P1/Evaluation/t10k-labels.idx1-ubyte')

train_images = train_images / 255.0
evaluation_images = evaluation_images / 255.0
train_images = train_images.reshape(-1, 28 * 28)
evaluation_images = evaluation_images.reshape(-1, 28 * 28)

train_labels = to_categorical(train_labels, num_classes=10)
evaluation_labels = to_categorical(evaluation_labels, num_classes=10)

In [ ]:
#Autoencoder:
input_img = Input(shape=(28 * 28,))
encoded = Dense(256, activation='relu')(input_img)
encoded = Dropout(0.2)(encoded)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dropout(0.2)(encoded)
encoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(28 * 28, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
# Training autoencoder:
autoencoder.fit(train_images, train_images, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

Epoch 1/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - loss: 0.0922 - val_loss: 0.0350
Epoch 2/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0345 - val_loss: 0.0242
Epoch 3/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0269 - val_loss: 0.0202
Epoch 4/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 0.0239 - val_loss: 0.0180
Epoch 5/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0220 - val_loss: 0.0165
Epoch 6/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.0209 - val_loss: 0.0154
Epoch 7/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0201 - val_loss: 0.0148
Epoch 8/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0193 - val_loss: 0.0140
Epoch 9/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0188 - val_loss: 0.0136
Epoch 10/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0182 - val_loss: 0.0131
Epoch 11/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0178 - val_loss: 0.0127
Epoch 12/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Neural network:
encoder = Model(input_img, encoded)
train_features = encoder.predict(train_images)
evaluation_features = encoder.predict(evaluation_images)

model = Sequential([
    Dense(128, activation='relu', input_shape=(64,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # Ajustar la tasa de aprendizaje
              loss='categorical_crossentropy', metrics=['accuracy'])



1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Training neural network:
model.fit(train_features, train_labels, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5187 - loss: 1.5483 - val_accuracy: 0.9197 - val_loss: 0.2876
Epoch 2/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8716 - loss: 0.4134 - val_accuracy: 0.9387 - val_loss: 0.2108
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9045 - loss: 0.3101 - val_accuracy: 0.9458 - val_loss: 0.1810
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9182 - loss: 0.2640 - val_accuracy: 0.9541 - val_loss: 0.1566
Epoch 5/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9260 - loss: 0.2363 - val_accuracy: 0.9572 - val_loss: 0.1429
Epoch 6/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9366 - loss: 0.2051 - val_accuracy: 0.9610 - val_loss: 0.1344
Epoch 7/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9398 - loss: 0.1959 - val_accuracy: 0.9616 - val_loss: 0.1270
Epoch 8/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9450 - loss: 0.1823 - val_accuracy: 0.

In [ ]:
# Train set :
_, train_accuracy = model.evaluate(train_features, train_labels, verbose=0)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

# Evaluation set:
_, evaluation_accuracy = model.evaluate(evaluation_features, evaluation_labels, verbose=0)
print(f"Evaluation Accuracy: {evaluation_accuracy * 100:.2f}%")

Train Accuracy: 98.60%
Evaluation Accuracy: 97.67%
